<a href="https://colab.research.google.com/github/KlaidasKaralevicius/NLP_lab_3/blob/main/lab3_Klaidas_Karalevicius.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -U transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
  Attempting uninstall: tr

In [4]:
import random
random.seed(22)

In [5]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
#import torch
dataset = load_dataset("xsum", trust_remote_code=True)
model_cpkt = 'facebook/bart-large-cnn'
tokenizer = AutoTokenizer.from_pretrained(model_cpkt)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/6.24k [00:00<?, ?B/s]

xsum.py:   0%|          | 0.00/5.76k [00:00<?, ?B/s]

(…)SUM-EMNLP18-Summary-Data-Original.tar.gz:   0%|          | 0.00/255M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

...

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [11]:
n_samples = 2000
small_train_dataset = dataset["train"].shuffle().select(range(n_samples))
small_eval_dataset = dataset["test"].shuffle().select(range(n_samples))
#small_test_dataset = dataset["test"].shuffle().select(range(n_samples))

In [12]:
def tokenize(text):
  return tokenizer(text['document'], truncation=True)

encoded_train = small_train_dataset.map(tokenize, batched=True)
encoded_eval = small_eval_dataset.map(tokenize, batched=True)
#encoded_test = small_test_dataset.map(tokenize, batched=True)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [9]:
target = encoded_train.features['summary']

TypeError: 'Value' object is not subscriptable

In [10]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_cpkt)

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [13]:
batch_size = 16

tf_train_dataset = model.prepare_tf_dataset(
    encoded_train,
    shuffle=True,
    batch_size=batch_size,
    tokenizer=tokenizer
)

tf_eval_dataset = model.prepare_tf_dataset(
    encoded_eval,
    shuffle=True,
    batch_size=batch_size,
    tokenizer=tokenizer
)

AttributeError: 'BartForConditionalGeneration' object has no attribute 'prepare_tf_dataset'

In [28]:
dialogue = '''The suspect in the shooting death of the UnitedHealthcare CEO, Brian Thompson, screamed: “This is completely out of touch and an insult to the intelligence of the American people” Tuesday before he fought being extradited from Pennsylvania to face murder charges in New York.

During the hearing, beside choosing to fight his extradition, Luigi Mangione was denied bail. For now, Mangione, 26, will return to custody at Pennsylvania’s Huntingdon state correctional institution while navigating the process to contest extradition, which could take days – if not a month or more, as the New York Times noted.
Mangione appeared for the proceeding in handcuffs and an orange prison jumpsuit, and video showed him yelling as police led him into the courthouse. Not all of what he screamed was immediately intelligible.

Mangione was arrested on Monday at a McDonald’s restaurant in Altoona, Pennsylvania, in connection with Thompson’s killing outside a Manhattan hotel hosting a meeting of investor’s in the CEO’s company.

The authorities in Pennsylvania detained him after being recognized by a local person from photographs released by New York police investigating the murder.

Police found Mangione with a firearm suppressor, a mask like the gunman’s, a fake New Jersey ID matching the one the police say the gunman used to check into the New York City hostel before the shooting, and a handwritten document, according to New York authorities.

On Monday night, Mangione was charged with second-degree murder, forgery and three gun charges by prosecutors in New York, following his arraignment at the Blair county courthouse in Altoona, Pennsylvania, where he faces separate charges of carrying a gun without a license, forgery, falsely identifying himself to the authorities and possessing “instruments of crime”.

During the arraignment hearing Monday, the judge asked Mangione if he understood the charges against him, and he said he did. No plea was entered and he was denied bail.

As detectives begin piecing together Mangione’s history, one area of focus has been the chronic back problems that he appears to have suffered since childhood.

Investigators could explore whether Mangione was denied insurance coverage for his condition.

He also appears to have withdrawn from social connections months earlier, prompting one to write on social media: “I don’t know if you are ok or just in a super isolated place … but I haven’t heard from you in months.”

The Mangione family released a statement on Monday evening, stating that the family was “shocked and devastated by Luigi’s arrest”.

“We offer our prayers to the family of Brian Thompson, and we ask people to pray for all involved,” they added.
'''

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Evaluate results: {'eval_loss': 2.580325126647949, 'eval_model_preparation_time': 0.0064, 'eval_runtime': 56.9634, 'eval_samples_per_second': 8.778, 'eval_steps_per_second': 0.562}


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

# Different

In [1]:
!pip install -U transformers datasets
import random
random.seed(22)
from datasets import load_dataset
import tensorflow as tf
import numpy as np
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM, create_optimizer

dataset = load_dataset("xsum", trust_remote_code=True)
model_cpkt = 'facebook/bart-large-cnn'
tokenizer = AutoTokenizer.from_pretrained(model_cpkt)
n_samples = 2000
small_train_dataset = dataset["train"].shuffle().select(range(n_samples))
small_eval_dataset = dataset["test"].shuffle().select(range(n_samples))
def tokenize(text):
  return tokenizer(text['document'], truncation=True)

encoded_train = small_train_dataset.map(tokenize, batched=True)
encoded_eval = small_eval_dataset.map(tokenize, batched=True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 395.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
  Attempting uninstall: 

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/6.24k [00:00<?, ?B/s]

xsum.py:   0%|          | 0.00/5.76k [00:00<?, ?B/s]

(…)SUM-EMNLP18-Summary-Data-Original.tar.gz:   0%|          | 0.00/255M [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [2]:
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_cpkt)

# Define batch size and number of epochs
batch_size = 4
num_epochs = 1

# Tokenize function for input and target pairs
def tokenize_summarization(examples):
    inputs = tokenizer(examples['document'], max_length=256, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['summary'], max_length=32, truncation=True, padding="max_length")
    inputs['labels'] = labels['input_ids']
    return inputs

# Prepare tokenized datasets
train_dataset = small_train_dataset.map(tokenize_summarization, batched=True)
eval_dataset = small_eval_dataset.map(tokenize_summarization, batched=True)

# Convert datasets to TensorFlow format
train_tf_dataset = tf.data.Dataset.from_tensor_slices((
    {key: np.array([example[key] for example in train_dataset]) for key in ['input_ids', 'attention_mask']},
    np.array([example['labels'] for example in train_dataset])
)).shuffle(buffer_size=len(train_dataset)).batch(batch_size)

eval_tf_dataset = tf.data.Dataset.from_tensor_slices((
    {key: np.array([example[key] for example in eval_dataset]) for key in ['input_ids', 'attention_mask']},
    np.array([example['labels'] for example in eval_dataset])
)).batch(batch_size)

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBartForConditionalGeneration.

All the weights of TFBartForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForConditionalGeneration for predictions without further training.


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [3]:
batches_per_epoch = len(train_dataset) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)

# Set up optimizer and learning rate schedule
optimizer, schedule = create_optimizer(
    init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps
)

In [4]:
model.compile(optimizer=optimizer, metrics=['accuracy'])

# Fine-tune the model
model.fit(
    train_tf_dataset,
    validation_data=eval_tf_dataset,
    epochs=num_epochs
)

500/500 [==============================] - 530s 799ms/step - loss: 2.0274 - accuracy: 0.4704 - val_loss: 1.9457 - val_accuracy: 0.4804


In [5]:
documents = '''The suspect in the shooting death of the UnitedHealthcare CEO, Brian Thompson, screamed: “This is completely out of touch and an insult to the intelligence of the American people” Tuesday before he fought being extradited from Pennsylvania to face murder charges in New York.

During the hearing, beside choosing to fight his extradition, Luigi Mangione was denied bail. For now, Mangione, 26, will return to custody at Pennsylvania’s Huntingdon state correctional institution while navigating the process to contest extradition, which could take days – if not a month or more, as the New York Times noted.
Mangione appeared for the proceeding in handcuffs and an orange prison jumpsuit, and video showed him yelling as police led him into the courthouse. Not all of what he screamed was immediately intelligible.

Mangione was arrested on Monday at a McDonald’s restaurant in Altoona, Pennsylvania, in connection with Thompson’s killing outside a Manhattan hotel hosting a meeting of investor’s in the CEO’s company.

The authorities in Pennsylvania detained him after being recognized by a local person from photographs released by New York police investigating the murder.

Police found Mangione with a firearm suppressor, a mask like the gunman’s, a fake New Jersey ID matching the one the police say the gunman used to check into the New York City hostel before the shooting, and a handwritten document, according to New York authorities.

On Monday night, Mangione was charged with second-degree murder, forgery and three gun charges by prosecutors in New York, following his arraignment at the Blair county courthouse in Altoona, Pennsylvania, where he faces separate charges of carrying a gun without a license, forgery, falsely identifying himself to the authorities and possessing “instruments of crime”.

During the arraignment hearing Monday, the judge asked Mangione if he understood the charges against him, and he said he did. No plea was entered and he was denied bail.

As detectives begin piecing together Mangione’s history, one area of focus has been the chronic back problems that he appears to have suffered since childhood.

Investigators could explore whether Mangione was denied insurance coverage for his condition.

He also appears to have withdrawn from social connections months earlier, prompting one to write on social media: “I don’t know if you are ok or just in a super isolated place … but I haven’t heard from you in months.”

The Mangione family released a statement on Monday evening, stating that the family was “shocked and devastated by Luigi’s arrest”.

“We offer our prayers to the family of Brian Thompson, and we ask people to pray for all involved,” they added.
'''

tokenized_inputs = tokenizer(documents, return_tensors="np", padding="longest", truncation=True, max_length=1024)
predictions = model.generate(
    input_ids=tokenized_inputs['input_ids'],
    attention_mask=tokenized_inputs['attention_mask'],
    max_length=128,
    num_beams=4,
    early_stopping=True
)

summaries = tokenizer.batch_decode(predictions, skip_special_tokens=True)
print(summaries)

['The suspect in the murder of Brian Thompson, the CEO of health insurance giant UnitedHealthcare, has been denied bail as he fights his extradition to New York to face charges of murder and other charges in the New York City shooting death of Thompson, who was shot in the head on Monday.']
